In [43]:
import pandas as pd
import numpy as np
%matplotlib inline 

In [44]:
# 2014 
df = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')
hh = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_household.tsv', sep='\t')
parcels = pd.read_csv(r'I:\T2040\soundcast_2014\inputs\accessibility\parcels_urbansim.txt', sep = ' ')

In [45]:

# AAA Calculations availavble for small, medium, large Sedans (and average of these), SUV, Minivan

# Analysis of survey could be used to allocate vehicle type distributions?

# http://publicaffairsresources.aaa.biz/wp-content/uploads/2016/03/2016-YDC-Brochure.pdf
# results are averages for sedans

# Annualalize costs as 262
annual_factor = 262

# Costs by miles
# Operating costs total (gas + maintenance + tries)
depreciation_per_mile = 3759.0/15000    # depreciation by mile

# Annual costs
insurance = 1222    #(full-coverage insurance, license, registration, taxes, depreciation @ 15,000 miles per year), finance charge
taxes = 687    # license, registration, taxes
finance = 683    # load financing

annual_fixed_costs = insurance+taxes+finance

In [46]:
#to calculate activity durations for parking cost:
day_minutes = 1440
minutes_hr = 60.0
cents_dollar = 100

In [47]:
# get total travel cost (based on vmt)
# includes 
hh_mile_costs = df.groupby('hhno').sum()[['travcost']]
hh_mile_costs = hh_mile_costs.reset_index()

hh = pd.merge(hh, hh_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh['travcost'] = hh['travcost'].fillna(0)

In [48]:
# Total vehicle miles traveled per household to calculate depreciation
driver_trips = df[df['dorp'] == 1]

In [49]:
hh_travdist = df.groupby('hhno').sum()[['travdist']]
hh_travdist = hh_travdist.reset_index()

In [50]:
hh = pd.merge(hh, hh_travdist, on='hhno', how='left')

In [51]:
# annual operating costs
driver_trips['operating_cost_drivers'] = driver_trips['travcost']*annual_factor
driver_trips['travdist'] = driver_trips['travdist'].fillna(0)
driver_trips['depreciation'] = driver_trips['travdist']*depreciation_per_mile*annual_factor

#calculating parking costs based on activity duration
driver_trips = pd.merge(driver_trips, parcels[['PARCELID','PPRICHRP']], left_on='dpcl', right_on = 'PARCELID')
# if a trip spans midnight you need to adjust across the day - like this:
driver_trips['duration'] = np.where(driver_trips['endacttm']<driver_trips['arrtm'], day_minutes-driver_trips['arrtm'] +driver_trips['endacttm'], driver_trips['endacttm']-driver_trips['arrtm'])/minutes_hr
driver_trips['parking_cost'] = (driver_trips['duration']*driver_trips['PPRICHRP']*annual_factor)/cents_dollar

driver_trips_hh = driver_trips.groupby('hhno').sum()[['operating_cost_drivers','depreciation', 'parking_cost']].reset_index()
hh = pd.merge(hh, driver_trips_hh, on = 'hhno')

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [52]:
# fixed costs as function of number of vehicles owned
hh['veh_insurance'] = hh['hhvehs']*insurance
hh['veh_taxes'] = hh['hhvehs']*taxes
hh['veh_finance'] = hh['hhvehs']*finance

hh['annual_auto_costs'] = hh['operating_cost_drivers']+hh['depreciation']+hh['veh_insurance']+hh['veh_taxes']+hh['veh_finance']+hh['parking_cost']
hh['annual_auto_costs']=hh['annual_auto_costs'].fillna(0)
# Need to add per mile depreciation costs

In [53]:
# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_travdist_transit_cost['transit_cost'] = hh_travdist_transit_cost['travcost']
hh_travdist_transit_cost = hh_travdist_transit_cost.reset_index()
hh_travdist_transit_cost['annual_transit_cost'] = hh_travdist_transit_cost['transit_cost']*annual_factor

In [54]:
hh = pd.merge(hh, hh_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')
hh['annual_transit_cost'] = hh['annual_transit_cost'].fillna(0)

In [55]:
hh['total_cost']= hh['annual_transit_cost']+hh['annual_auto_costs']
hh = hh.loc[hh['hhincome']!=0]
hh['percent_transport_cost'] = hh['total_cost']/hh['hhincome']
hh['percent_transport_cost'] =hh['percent_transport_cost'].fillna(0)

In [59]:
print hh['total_cost'].mean()
print hh['parking_cost'].mean()
print hh['veh_insurance'].mean()
print hh['veh_taxes'].mean()
print hh['operating_cost_drivers'].mean()
print hh['depreciation'].mean()
print hh['annual_transit_cost'].mean()
print hh['percent_transport_cost'].median()

11466.6519134
434.854080017
2466.04274411
1386.39227922
2586.48255642
3139.09778208
75.4623453191
0.138131255974
